# CS 4650 Final Project

#### The second iteration of this project uses the Yahoo API to get live market data and use a simple bollinger band based algorithm that is very simple, if the stock prices rises or is lower than 2 standard deviations away from the price then you buy or sell. That is the essential premise of this new algorithm that I am using.

In [13]:
pip install plotly

SyntaxError: invalid syntax. Perhaps you forgot a comma? (Temp/ipykernel_5712/3695751552.py, line 1)

In [14]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install --upgrade nbformat

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy
import pandas as pd

import yfinance as yf
import plotly.graph_objs as go

import time

In [2]:
#The Yahoo API needs the following inputs: Tickers, Start and End Date (or Period), and Interval
# period is 1 week for this example and interval is one minute from which we will get our data
Data = yf.download(tickers = 'NVDA', period = '1d', interval = '1m')

#Then we print the data
Data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-11-26 09:30:00-05:00,326.000000,326.700012,324.000092,326.329987,326.329987,2241269
2021-11-26 09:31:00-05:00,326.910004,327.100006,326.019989,326.470001,326.470001,267658
2021-11-26 09:32:00-05:00,326.420013,326.570007,323.290009,323.484985,323.484985,193962
2021-11-26 09:33:00-05:00,324.500000,325.109985,324.479004,324.967987,324.967987,189511
2021-11-26 09:34:00-05:00,326.000000,326.177002,325.969696,326.084991,326.084991,189397
...,...,...,...,...,...,...
2021-11-26 12:56:00-05:00,316.055603,316.547302,315.920013,316.000000,316.000000,163103
2021-11-26 12:57:00-05:00,315.899994,317.079987,315.890015,316.380005,316.380005,216880
2021-11-26 12:58:00-05:00,316.320007,316.899902,316.000000,316.174988,316.174988,267509


In [5]:
while(1):
    #The Yahoo API needs the following inputs: Tickers, Start and End Date (or Period), and Interval
    # period is 1 week for this example and interval is one minute from which we will get our data
    stock = 'AAPL'
    Data = yf.download(tickers = stock, period = '1d', interval = '1m')

    #Create a figure
    fig = go.Figure()

    #We will now emplpy the Bollinger Band Strategy Here

    #Middle Band will be calculated with the following calculation: MiddleBand(t) = Average( Close(t-1 to t-22) ), we will use the rolling function for volatility
    #For the Upper and Lower Band we will go 2 standard deviations away from the middle band but we will use the rolling function to account for volatility

    #Calculate the middle, lower, and upper band
    Data['Middle Band'] = Data['Close'].rolling(window=21).mean()
    Data['Upper Band'] = Data['Middle Band'] + 1.96*Data['Close'].rolling(window=21).std()
    Data['Lower Band'] = Data['Middle Band'] - 1.96*Data['Close'].rolling(window=21).std()

    #Now we draw the Middle, Lower, and Upper Bands into the figure
    fig.add_trace(go.Scatter(x = Data.index, y = Data['Middle Band'], line = dict(color = 'blue', width = 0.7), name = 'Middle Band'))
    fig.add_trace(go.Scatter(x = Data.index, y = Data['Upper Band'], line = dict(color = 'red', width = 0.7), name = 'Upper Band (Sell)'))
    fig.add_trace(go.Scatter(x = Data.index, y = Data['Lower Band'], line = dict(color = 'green', width = 0.7), name = 'Lower Band (Buy)'))

    #Create Candlesticks
    fig.add_trace(go.Candlestick(x = Data.index, open = Data['Open'], high = Data['High'], low = Data['Low'], close = Data['Close'], name = 'Market Data'))

    #Add Titles to the Chart
    #Modify Y-Axis and Add a Title
    fig.update_layout(
                    title = stock + ' Live Share Price vs. Time',
                    yaxis_title = 'Stock Price (USD/Share)'
                        )

    #Modify X-Axis
    fig.update_xaxes(
        rangeslider_visible = True, 
        rangeselector = dict(
            buttons = list([
                            dict(count=15, label="15m", step="minute", stepmode="backward"),
                            dict(count=45, label="45m", step="minute", stepmode="backward"),
                            dict(count=1, label="HTD", step="hour", stepmode="todate"),
                            dict(count=3, label="3h", step="hour", stepmode="backward"),
                            dict(step="all")

                            ])
                        )
                    )

    #Display Figure
    fig.show()

    #Delay the refresh rate to only every minute
    time.sleep(20)

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
